In [4]:
!python -V

Python 3.7.4


# 定义 intent 和 slot

In [1]:
intent = ['看不懂的','闲聊','商品/品类','搜优惠','搜活动/会场','其他']
slot = ['日期','商品名','品牌','店铺','颜色','价格','数量','属性']
slot_dict = {'日期':'DATE','商品/品类':'PRODUCT','品牌':'BRAND','店铺':'SHOP','颜色':'COLOR','价格':'PRICE','数量':'AMOUT','属性':'ATTRIBUTE'}

In [2]:
#dict_1 = {'B-DATE': 0, 'B-PRODUCT': 1, 'B-BRAND': 2, 'B-SHOP': 3, 'B-COLOR': 4, 'B-PRICE': 5, 'B-AMOUT': 6, 'B-ATTRIBUTE': 7, 'I-DATE': 8, 'I-PRODUCT': 9, 'I-BRAND': 10, 'I-SHOP': 11, 'I-COLOR': 12, 'I-PRICE': 13, 'I-AMOUT': 14, 'I-ATTRIBUTE': 15, 'E-DATE': 16, 'E-PRODUCT': 17, 'E-BRAND': 18, 'E-SHOP': 19, 'E-COLOR': 20, 'E-PRICE': 21, 'E-AMOUT': 22, 'E-ATTRIBUTE': 23, 'O-DATE': 24, 'O-PRODUCT': 25, 'O-BRAND': 26, 'O-SHOP': 27, 'O-COLOR': 28, 'O-PRICE': 29, 'O-AMOUT': 30, 'O-ATTRIBUTE': 31, 'S-DATE': 32, 'S-PRODUCT': 33, 'S-BRAND': 34, 'S-SHOP': 35, 'S-COLOR': 36, 'S-PRICE': 37, 'S-AMOUT': 38, 'S-ATTRIBUTE': 39, '<PAD>': 40, 'O': 41}  
dict_1 = {'B-DATE': 0, 'B-PRODUCT': 1, 'B-BRAND': 2, 'B-SHOP': 3, 'B-COLOR': 4, 'B-PRICE': 5, 'B-AMOUT': 6, 'B-ATTRIBUTE': 7, 'I-DATE': 8, 'I-PRODUCT': 9, 'I-BRAND': 10, 'I-SHOP': 11, 'I-COLOR': 12, 'I-PRICE': 13, 'I-AMOUT': 14, 'I-ATTRIBUTE': 15, 'E-DATE': 16, 'E-PRODUCT': 17, 'E-BRAND': 18, 'E-SHOP': 19, 'E-COLOR': 20, 'E-PRICE': 21, 'E-AMOUT': 22, 'E-ATTRIBUTE': 23, 'S-DATE': 24, 'S-PRODUCT': 25, 'S-BRAND': 26, 'S-SHOP': 27, 'S-COLOR': 28, 'S-PRICE': 29, 'S-AMOUT': 30, 'S-ATTRIBUTE': 31, '<PAD>': 32, 'O': 33}
{0: 'B-DATE', 1: 'B-PRODUCT', 2: 'B-BRAND', 3: 'B-SHOP', 4: 'B-COLOR', 5: 'B-PRICE', 6: 'B-AMOUT', 7: 'B-ATTRIBUTE', 8: 'I-DATE', 9: 'I-PRODUCT', 10: 'I-BRAND', 11: 'I-SHOP', 12: 'I-COLOR', 13: 'I-PRICE', 14: 'I-AMOUT', 15: 'I-ATTRIBUTE', 16: 'E-DATE', 17: 'E-PRODUCT', 18: 'E-BRAND', 19: 'E-SHOP', 20: 'E-COLOR', 21: 'E-PRICE', 22: 'E-AMOUT', 23: 'E-ATTRIBUTE', 24: 'S-DATE', 25: 'S-PRODUCT', 26: 'S-BRAND', 27: 'S-SHOP', 28: 'S-COLOR', 29: 'S-PRICE', 30: 'S-AMOUT', 31: 'S-ATTRIBUTE', 32: '<PAD>', 33: 'O'}
print(dict_1)
print(type(dict_1))
for key, value in dict_1.items():
    print(key+'\t'+str(value))

{'B-DATE': 0, 'B-PRODUCT': 1, 'B-BRAND': 2, 'B-SHOP': 3, 'B-COLOR': 4, 'B-PRICE': 5, 'B-AMOUT': 6, 'B-ATTRIBUTE': 7, 'I-DATE': 8, 'I-PRODUCT': 9, 'I-BRAND': 10, 'I-SHOP': 11, 'I-COLOR': 12, 'I-PRICE': 13, 'I-AMOUT': 14, 'I-ATTRIBUTE': 15, 'E-DATE': 16, 'E-PRODUCT': 17, 'E-BRAND': 18, 'E-SHOP': 19, 'E-COLOR': 20, 'E-PRICE': 21, 'E-AMOUT': 22, 'E-ATTRIBUTE': 23, 'S-DATE': 24, 'S-PRODUCT': 25, 'S-BRAND': 26, 'S-SHOP': 27, 'S-COLOR': 28, 'S-PRICE': 29, 'S-AMOUT': 30, 'S-ATTRIBUTE': 31, '<PAD>': 32, 'O': 33}
<class 'dict'>
B-DATE	0
B-PRODUCT	1
B-BRAND	2
B-SHOP	3
B-COLOR	4
B-PRICE	5
B-AMOUT	6
B-ATTRIBUTE	7
I-DATE	8
I-PRODUCT	9
I-BRAND	10
I-SHOP	11
I-COLOR	12
I-PRICE	13
I-AMOUT	14
I-ATTRIBUTE	15
E-DATE	16
E-PRODUCT	17
E-BRAND	18
E-SHOP	19
E-COLOR	20
E-PRICE	21
E-AMOUT	22
E-ATTRIBUTE	23
S-DATE	24
S-PRODUCT	25
S-BRAND	26
S-SHOP	27
S-COLOR	28
S-PRICE	29
S-AMOUT	30
S-ATTRIBUTE	31
<PAD>	32
O	33


In [3]:
import pandas 
import os
def read_data_by_folder(folder):
    text_list = []
    slot_list = []
    
    file_list = os.listdir(folder)
    for file_item in file_list:
        current_file_item = os.path.join(folder, file_item)
        data = pandas.read_csv(current_file_item)
        for index in data.index:
            tmp = data.loc[index].values
            text_list.append(tmp[0])

            tmp_slot_list = []
            for idx, value in enumerate(tmp):
                if idx>2 and idx %2 == 1 and idx < len(tmp)-2:
                    slot_type = tmp[idx]
                    slot_value = tmp[idx+1]
                    if len(str(slot_type).strip()) > 1:
                        try:
                            tmp_slot_list.append((slot_dict[slot_type], slot_value))
                        except:
                            tmp_slot_list.append(('ATTRIBUTE', slot_value))
            slot_list.append(tmp_slot_list)
    return text_list, slot_list

def read_clf_data_by_folder(folder):
    clf_list = []
    file_list = os.listdir(folder)
    for file_item in file_list:
        current_file_item = os.path.join(folder, file_item)
        data = pandas.read_csv(current_file_item)
        for index in data.index:
            tmp = data.loc[index].values
            clf_list.append(tmp[1]+ '\t' +tmp[0])
       
    return clf_list

# 处理slot

In [0]:
# 读 数据
text_list, slot_list = read_data_by_folder("/Users/sunhongchao/Desktop/Projects/1111/dataset/predict/")
# 写 数据
output = open('label.txt', encoding='utf-8', mode='w')

for text, tmp_slot_list in zip(text_list, slot_list):
    text_label = ['O']*len(text)
    origin_text_label = text_label.copy()
    for slot in tmp_slot_list:
        idx = text.index(slot[1])
        if idx < 0:
            continue
        slot_type = slot[0]
        
        if len(slot[1]) == 1:
            text_label[idx] = "S" + "-" + slot_type
        else:
            text_label[idx] = "B" + "-" + slot_type
        
            for idx in range(idx+1, idx+len(slot[1])-1, 1):
                text_label[idx] = "I" + "-" + slot_type
            
            text_label[idx+1] = "E" + "-" + slot_type
    
    import operator
    if operator.eq(text_label, origin_text_label) is True:
        continue

    for char, label in zip(text, text_label):
        output.write(char + '\t' + label + '\n')
    output.write('\n')
    
output.close()

# 处理意图

In [18]:
import pandas 
from sklearn.model_selection import train_test_split
import random

clf_list = read_clf_data_by_folder("/Users/sunhongchao/Desktop/Projects/1111/dataset/all_data/")
output_list = []

for item in clf_list:
    cut_list = item.split("\t")
    if len(cut_list) != 2:
        continue
    if cut_list[0] in ["其它属性", "看不懂的"]:
        continue 
    output_list.append(item)

clf_list = output_list

random.shuffle(clf_list)

cut_idx = int(len(clf_list) * 0.9)

train_clf_list = clf_list[:cut_idx]
test_clf_list = clf_list[cut_idx:]

clf_output = open('/Users/sunhongchao/Documents/craft/Workspace-of-NLU/corpus/nlu/promotion/train_clf.txt', encoding='utf-8', mode='w')
for item in train_clf_list:
    clf_output.write(item + '\n')

clf_output = open('/Users/sunhongchao/Documents/craft/Workspace-of-NLU/corpus/nlu/promotion/test_clf.txt', encoding='utf-8', mode='w')
for item in test_clf_list:
    clf_output.write(item + '\n')

# 数据扩充与优化方法间的关系

# Data augement for Unbalance NLU

In [0]:
import pandas 
from sklearn.model_selection import train_test_split
import random

clf_list = \
read_clf_data_by_folder("/Users/sunhongchao/Desktop/Projects/1111/dataset/all_data/")

clf_output = open('clf_promotion_activity.txt', encoding='utf-8', mode='w')
for item in clf_list:
    cut_list = item.split("\t")
    if len(cut_list) != 2:
        continue
    if cut_list[0] not in ["搜优惠", "搜活动/会场"]:
        continue 
    clf_output.write( item + '\n')


In [0]:
import synonyms
similarity_result = synonyms.nearby("明天")
print(similarity_result[0])
from snownlp import SnowNLP 
for item in similarity_result[0]:                                                   
    snow_result = SnowNLP(item)                                     
    print(' '.join('%s/%s' % (word, tag) for (word, tag) in snow_result.tags))

similarity_result = synonyms.nearby("优惠")
print(similarity_result[0])
from snownlp import SnowNLP 
for item in similarity_result[0]:                                                   
    snow_result = SnowNLP(item)                                     
    print(' '.join('%s/%s' % (word, tag) for (word, tag) in snow_result.tags))

In [6]:
def fun_similarity_words(word):

    import synonyms
    similarity_result = synonyms.nearby(word)
    from snownlp import SnowNLP 
    words_list = []
    pos_list = []
    return_words_list = []
    for item in similarity_result[0]:                                                   
        snow_result = SnowNLP(item)
        if len(words_list) is 0:
            words_list.append(' '.join('%s' % word for (word, tag) in snow_result.tags))
            pos_list.append(' '.join('/%s' % tag for (word, tag) in snow_result.tags))
        else:
            word = ' '.join('%s' % word for (word, tag) in snow_result.tags)
            pos =  ' '.join('/%s' % tag for (word, tag) in snow_result.tags)

            if pos == pos_list[0]:
                return_words_list.append(word)


    return return_words_list

print(fun_similarity_words("明天"))



smart_open library not found; falling back to local-filesystem-only
>> Synonyms load wordseg dict [/Users/sunhongchao/anaconda3/envs/tf_base/lib/python3.7/site-packages/synonyms/data/vocab.txt] ... 
>> Synonyms on loading stopwords [/Users/sunhongchao/anaconda3/envs/tf_base/lib/python3.7/site-packages/synonyms/data/stopwords.txt] ...
>> Synonyms on loading vectors [/Users/sunhongchao/anaconda3/envs/tf_base/lib/python3.7/site-packages/synonyms/data/words.vector] ...
['明日', '昨天', '今晚']


' '.join('%s/%s' % (word, tag) for (word, tag) in snow_result.tags))

similarity_result = synonyms.nearby("优惠")
print(similarity_result[0])


In [14]:
!pwd

/Users/sunhongchao/Documents/craft


In [17]:
from snownlp import SnowNLP 

# output_file = open('Workspce-of-NLU/corpus/nlu/promotion/clf_chat_augement.txt', mode='w', encoding='utf-8')

output_file = open('/Users/sunhongchao/Documents/craft/Workspace-of-NLU/corpus/nlu/promotion/clf_chat_augement.txt', mode='w', encoding='utf-8') 

with open('/Users/sunhongchao/Documents/craft/Workspace-of-NLU/corpus/nlu/promotion/clf_chat.txt', mode='r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        # print("*" *50)
        # print(line)
        snow_result = SnowNLP("".join(line.split("\t")[1:])) # 读取原始文本
        origin_tag = line.split("\t")[0]
        # print("before generation")                             
        # print(' '.join('%s/%s' % (word, tag) for (word, tag) in snow_result.tags))

        # print("\nafter generation")
        words = ' '.join('%s' % word for (word, tag) in snow_result.tags)
        word_list = words.split()

        for idx, word in enumerate(word_list):
            sim_word_list = fun_similarity_words(word)
            if len(sim_word_list) is 0:
                continue
             
            # print(word + ": similarity words list :", sim_word_list)

            # print("\n output results")
            for sim_word in sim_word_list:
                if idx != len(word_list)-1:
                    tmp_str="".join(word_list[:idx]) + sim_word +  "".join(word_list[idx+1:])
                    # print(tmp_str)
                    output_file.write(origin_tag + '\t' + tmp_str + '\n')

                else:
                    tmp_str = "".join(word_list[:idx]) + sim_word
                    # print(tmp_str)
                    output_file.write(origin_tag + '\t' + tmp_str + '\n')
                        
output_file.close()

# 多数类降采样

In [1]:
! conda install imblearn

Colledone
Solving environment:failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - imblearn

Current channels:

  - https://repo.anaconda.com/pkgs/main/osx-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/osx-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [28]:
text = "'<PAD>': 0, 'B-DATE': 1, 'B-PRODUCT': 2, 'B-BRAND': 3, 'B-SHOP': 4, 'B-COLOR': 5, 'B-PRICE': 6, 'B-AMOUT': 7, 'B-ATTRIBUTE': 8, 'I-DATE': 9, 'I-PRODUCT': 10, 'I-BRAND': 11, 'I-SHOP': 12, 'I-COLOR': 13, 'I-PRICE': 14, 'I-AMOUT': 15, 'I-ATTRIBUTE': 16, 'E-DATE': 17, 'E-PRODUCT': 18, 'E-BRAND': 19, 'E-SHOP': 20, 'E-COLOR': 21, 'E-PRICE': 22, 'E-AMOUT': 23, 'E-ATTRIBUTE': 24, 'S-DATE': 25, 'S-PRODUCT': 26, 'S-BRAND': 27, 'S-SHOP': 28, 'S-COLOR': 29, 'S-PRICE': 30, 'S-AMOUT': 31, 'S-ATTRIBUTE': 32, 'O': 33"
cut_list = text.split(",")
for item in cut_list:
    cut_item = item.split(":")
    tmp_item = cut_item[0].strip("'")
    tmp_item = tmp_item.lstrip("")
    print(tmp_item + '\t' + cut_item[1])

<PAD>	 0
 'B-DATE	 1
 'B-PRODUCT	 2
 'B-BRAND	 3
 'B-SHOP	 4
 'B-COLOR	 5
 'B-PRICE	 6
 'B-AMOUT	 7
 'B-ATTRIBUTE	 8
 'I-DATE	 9
 'I-PRODUCT	 10
 'I-BRAND	 11
 'I-SHOP	 12
 'I-COLOR	 13
 'I-PRICE	 14
 'I-AMOUT	 15
 'I-ATTRIBUTE	 16
 'E-DATE	 17
 'E-PRODUCT	 18
 'E-BRAND	 19
 'E-SHOP	 20
 'E-COLOR	 21
 'E-PRICE	 22
 'E-AMOUT	 23
 'E-ATTRIBUTE	 24
 'S-DATE	 25
 'S-PRODUCT	 26
 'S-BRAND	 27
 'S-SHOP	 28
 'S-COLOR	 29
 'S-PRICE	 30
 'S-AMOUT	 31
 'S-ATTRIBUTE	 32
 'O	 33


In [1]:
# 品牌检索

In [1]:
output_file = open('/Users/sunhongchao/Documents/craft/Workspace-of-NLU/corpus/nlu/promotion/brand_output.txt', mode='w', encoding='utf-8') 

brand_set = []
with open('/Users/sunhongchao/Documents/craft/Workspace-of-NLU/corpus/nlu/promotion/brand.txt', mode='r', encoding='utf-8') as f:
    lines = f.readlines()
    lines = [line.strip("\n") for line in lines if len(line) > 3 and len(line)< 6]
    brand_set = list(set(lines)) 
    print(len(brand_set))
    print(brand_set[:10])

output_brand_list = []


with open('/Users/sunhongchao/Documents/craft/Workspace-of-NLU/train_clf.txt', mode='r', encoding='utf-8') as f:
    lines = f.readlines()
    lines = list(set(lines))
    print(len(lines))



    for idx, line in enumerate(lines):
        if idx % 100 == 0:
            print(idx)
        for brand in brand_set:
            if len(brand) > 1 and len(brand) < 8:
                if brand in line:
                    output_brand_list.append(brand)
from collections import Counter
result_list = Counter(output_brand_list).most_common()

print(result_list)



0
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18500
18600
18700
18800
18900
19000
19100
19200
19300
19400
19500
19600
19700
19800
19900
20000
20100
20200
20300
20400
20500
20600
20700
20800
20900
21000
21100
21200
21300
21400
21500
21600
21700
21800
21900
22000
22100
22200
22300
22400
22500
22600
22700
22800
22900
23000
23100
23200
23300
23400
23500
23600
23700
23800
23900
24000
[('我想要', 343), ('双11', 266), ('100', 231), ('OPPO', 209), ('OPP', 209), ('机器人', 185), ('vivo', 128), ('viv', 128), ('200', 107), ('双十一', 102), ('阿迪达斯', 96), ('我要的', 84), ('飞利浦', 70), ('苏泊尔', 58), ('奥特曼', 53), ('n

In [2]:
for item in result_list:
    output_file.write(str(item[0]) + "\n")

In [ ]:
# 停用词
with open('/Users/sunhongchao/Documents/craft/Workspace-of-NLU/corpus/nlu/promotion/stop_words.txt', mode='r', encoding='utf-8') as f:
    lines = f.readlines()
    stop_words = list(set(lines))
    print(len(lines))

with open('/Users/sunhongchao/Documents/craft/Workspace-of-NLU/corpus/nlu/promotion/brand_output_all.txt', mode='r', encoding='utf-8') as f:
    lines = f.readlines()
    brand_all = list(set(lines))
    print(len(lines))

output_file = open('/Users/sunhongchao/Documents/craft/Workspace-of-NLU/corpus/nlu/promotion/brand_output_stopwords.txt', mode='w', encoding='utf-8') 

new_result_list = [item  for item in result_list if item not in stop_words]
print(result_list)
for item in new_result_list:
    output_file.write(str(item) + "\n")

In [4]:
brand_set = []
with open('/Users/sunhongchao/Documents/craft/Workspace-of-NLU/corpus/nlu/promotion/brand.txt', mode='r', encoding='utf-8') as f:
    lines = f.readlines()
    lines = [line.strip("\n") for line in lines if len(line) > 5 and len(line)< 8]
    brand_set = list(set(lines)) 
    print(len(brand_set))
    print(brand_set[:10])

myList1 = sorted(brand_set,key = lambda i:len(i),reverse=True)  
print(len(myList1))

26698
['LEEMIN', 'gonna', 'Kunodi', 'Vlush', '游子慈母酿', 'Belif', '青顶山庄园', '新世紀漢方', 'Tudor', 'eRouge']
26698
